# **1.6 Introduction to Graph Convolutions**

In [1]:
!pip install --pre deepchem

그래프 CNN: 원자(Node)와 결합(Edge)으로 이루어진 그래프 데이터를 훑음

 각 원자가 주변 원자들과 정보를 주고받으며(Message Passing), 스스로 학습

In [ ]:
import deepchem as dc

tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets

In [4]:
n_tasks = len(tasks)
num_features = train_dataset.X[0].get_atom_features().shape[1]
model = dc.models.torch_models.GraphConvModel(n_tasks, mode='classification',number_input_features=[num_features,64], device='cpu')
model.fit(train_dataset, nb_epoch=3)

0.8767154136400545

In [5]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
print('Training set score:', model.evaluate(train_dataset, [metric], transformers))
print('Test set score:', model.evaluate(test_dataset, [metric], transformers))

Training set score: {'roc_auc_score': 0.8336448743423391}
Test set score: {'roc_auc_score': 0.6521238707962081}


In [6]:
import torch
import torch.nn as nn
from deepchem.models.torch_models.layers import GraphConv, GraphGather, GraphPool

In [33]:
node_features = train_dataset.X[0].get_atom_features()
num_input_features = node_features.shape[1]
print(f"Number of input features: {num_input_features}")

Number of input features: 75


In [34]:
from deepchem.models.layers import GraphConv, GraphPool, GraphGather
import tensorflow as tf
import tensorflow.keras.layers as layers

batch_size = 100

class GraphConvModelTensorflow(tf.keras.Model):

  def __init__(self):
    super(GraphConvModelTensorflow, self).__init__()
    self.gc1 = GraphConv(128, activation_fn=tf.nn.tanh)
    self.batch_norm1 = layers.BatchNormalization()
    self.gp1 = GraphPool()

    self.gc2 = GraphConv(128, activation_fn=tf.nn.tanh)
    self.batch_norm2 = layers.BatchNormalization()
    self.gp2 = GraphPool()

    self.dense1 = layers.Dense(256, activation=tf.nn.tanh)
    self.batch_norm3 = layers.BatchNormalization()
    self.readout = GraphGather(batch_size=batch_size, activation_fn=tf.nn.tanh)

    self.dense2 = layers.Dense(n_tasks*2)
    self.logits = layers.Reshape((n_tasks, 2))
    self.softmax = layers.Softmax()

  def call(self, inputs):
    gc1_output = self.gc1(inputs)
    batch_norm1_output = self.batch_norm1(gc1_output)
    gp1_output = self.gp1([batch_norm1_output] + inputs[1:])

    gc2_output = self.gc2([gp1_output] + inputs[1:])
    batch_norm2_output = self.batch_norm1(gc2_output)
    gp2_output = self.gp2([batch_norm2_output] + inputs[1:])

    dense1_output = self.dense1(gp2_output)
    batch_norm3_output = self.batch_norm3(dense1_output)
    readout_output = self.readout([batch_norm3_output] + inputs[1:])

    logits_output = self.logits(self.dense2(readout_output))
    return self.softmax(logits_output)

In [42]:
model = dc.models.KerasModel(GraphConvModelTensorflow(), loss=dc.models.losses.CategoricalCrossEntropy())

In [43]:
test_dataset.X[0]

In [47]:
from deepchem.metrics import to_one_hot
from deepchem.feat.mol_graphs import ConvMol
import numpy as np

def data_generator(dataset, epochs=1):
  for ind, (X_b, y_b, w_b, ids_b) in enumerate(dataset.iterbatches(batch_size, epochs,
                                                                   deterministic=False, pad_batches=True)):
    multiConvMol = ConvMol.agglomerate_mols(X_b)
    inputs = [multiConvMol.get_atom_features(), multiConvMol.deg_slice, np.array(multiConvMol.membership)]
    for i in range(1, len(multiConvMol.get_deg_adjacency_lists())):
      inputs.append(multiConvMol.get_deg_adjacency_lists()[i])
    labels = [to_one_hot(y_b.flatten(), 2).reshape(-1, n_tasks, 2)]
    weights = [w_b]
    yield (inputs, labels, weights)

In [48]:
import torch
import torch.nn as nn
from deepchem.models.torch_models.layers import GraphConv, GraphGather, GraphPool

In [49]:
sample_batch = next(data_generator(train_dataset))
node_features = sample_batch[0][0]
num_input_features = node_features.shape[1]
print(f"Number of input features: {num_input_features}")

Number of input features: 75


In [50]:
class GraphConvModelTorch(nn.Module):
    def __init__(self):
        super(GraphConvModelTorch, self).__init__()

        self.gc1 = GraphConv(out_channel=128, number_input_features=num_input_features, activation_fn=nn.Tanh())
        self.batch_norm1 = nn.BatchNorm1d(128)
        self.gp1 = GraphPool()

        self.gc2 = GraphConv(out_channel=128, number_input_features=128, activation_fn=nn.Tanh())
        self.batch_norm2 = nn.BatchNorm1d(128)
        self.gp2 = GraphPool()

        self.dense1 = nn.Linear(128, 256)
        self.act3 = nn.Tanh()
        self.batch_norm3 = nn.BatchNorm1d(256)
        self.readout = GraphGather(batch_size=batch_size, activation_fn=nn.Tanh())

        self.dense2 = nn.Linear(512, n_tasks * 2)

        self.logits = lambda data: data.view(-1, n_tasks, 2)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, inputs):
        gc1_output = self.gc1(inputs)
        batch_norm1_output = self.batch_norm1(gc1_output)
        gp1_output = self.gp1([batch_norm1_output] + inputs[1:])

        gc2_output = self.gc2([gp1_output] + inputs[1:])
        batch_norm2_output = self.batch_norm2(gc2_output)
        gp2_output = self.gp2([batch_norm2_output] + inputs[1:])

        dense1_output = self.act3(self.dense1(gp2_output))
        batch_norm3_output = self.batch_norm3(dense1_output)
        readout_output = self.readout([batch_norm3_output] + inputs[1:])

        dense2_output = self.dense2(readout_output)
        logits_output = self.logits(dense2_output)
        softmax_output = self.softmax(logits_output)
        return softmax_output

In [52]:
model = dc.models.TorchModel(GraphConvModelTorch(), loss=dc.models.losses.CategoricalCrossEntropy())
model.fit_generator(data_generator(train_dataset, epochs=3))

0.8564493415061007

In [53]:
print('Training set score:', model.evaluate_generator(data_generator(train_dataset), [metric], transformers))
print('Test set score:', model.evaluate_generator(data_generator(test_dataset), [metric], transformers))

Training set score: {'roc_auc_score': 0.8556463164250768}
Test set score: {'roc_auc_score': 0.6597936112665707}


# 💡 Insight & Reflection

GraphConv(정보 교환) → GraphPool(정보 압축) → GraphGather(분자 단위 통합) 레이어를 거치며, 개별 원자의 정보가 고차원적인 분자/결정 전체의 특성으로 진화하는 메커니즘

모델이 원자 간의 연결 관계(Graph)를 직접 훑으며 물성 결정에 핵심적인 화학적 특징을 스스로 추출

원자의 배열이 물성을 결정하는 반도체 격자나 고분자 구조 분석에서, 미세한 결함이나 구조적 뒤틀림이 전체 시스템에 미치는 영향을 파악하는 데 가장 적합한 아키텍처